In [22]:
import sys
import os
import pandas as pd

from football_predictions.data.tools.rolling_averages import rolling_averages_for_window_sizes
from football_predictions.data.configuration import ROLLING_AVERAGES_COLUMNS, ROLLING_AVERAGES_WINDOW_SIZES
from football_predictions.data.tools.processed import calculate_rolling_averages_home_team, calculate_rolling_averages_away_team

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [5]:
data = pd.read_csv('../data/interim/premier_league/premier_league_2324.csv')
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HomeTeamCode,AwayTeamCode,FTR_code,HTR_code
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,1.95,1.97,NaN,NaN,1.92,1.95,5,12,-1,-1
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,1.93,1.97,2.01,2.09,1.95,1.92,0,15,1,1
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.01,1.92,2.06,1.96,1.96,1.91,2,18,0,0
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.00,1.91,2.14,1.93,2.00,1.86,4,11,1,1
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,2.04,1.88,2.08,1.99,1.98,1.88,8,9,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,19/05/2024,16:00,Crystal Palace,Aston Villa,5,0,H,2,0,...,1.78,2.16,1.93,2.21,1.81,2.05,7,1,1,1
376,E0,19/05/2024,16:00,Liverpool,Wolves,2,0,H,2,0,...,2.04,1.85,2.10,1.89,2.04,1.82,10,19,1,1
377,E0,19/05/2024,16:00,Luton,Fulham,2,4,A,1,2,...,1.99,1.93,2.02,1.94,1.96,1.91,11,9,-1,-1
378,E0,19/05/2024,16:00,Man City,West Ham,3,1,H,2,1,...,1.99,1.90,2.05,1.99,1.96,1.91,12,18,1,1


In [15]:
grouped_home = data.groupby("HomeTeam").apply(
    lambda x:
        rolling_averages_for_window_sizes(
            x,
            ROLLING_AVERAGES_COLUMNS,
            ROLLING_AVERAGES_WINDOW_SIZES,
            'H'
            )
        )
grouped_home

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_3727/2705378691.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_home = data.groupby("HomeTeam").apply(


Div        Date   Time HomeTeam          AwayTeam  FTHG  FTAG  \
HomeTeam                                                                     
Arsenal  21   E0  26/08/2023  15:00  Arsenal            Fulham     2     2   
         54   E0  24/09/2023  14:00  Arsenal         Tottenham     2     2   
         92   E0  28/10/2023  15:00  Arsenal  Sheffield United     5     0   
         165  E0  17/12/2023  14:00  Arsenal          Brighton     2     0   
         187  E0  28/12/2023  20:15  Arsenal          West Ham     0     2   
...           ..         ...    ...      ...               ...   ...   ...   
Wolves   192  E0  30/12/2023  15:00   Wolves           Everton     3     0   
         256  E0  25/02/2024  13:30   Wolves  Sheffield United     1     0   
         325  E0  20/04/2024  19:30   Wolves           Arsenal     0     2   
         331  E0  24/04/2024  19:45   Wolves       Bournemouth     0     1   
         340  E0  27/04/2024  15:00   Wolves             Luton     2     1   

             FTR  HTHG  HTAG  ... H_R_HC_W_4 H_R_AC_W_4  H_R_HF_W_4  \
HomeTeam                      ...                                     
Arsenal  21    D     0     1  ...       7.50       3.25       11.75   
         54    D     1     1  ...       6.75       2.50       10.75   
         92    H     1     0  ...       8.00       2.25       11.25   
         165   H     0     0  ...       6.25       4.00       10.25   
         187   A     0     1  ...       8.50       2.00       10.50   
...           ..   ...   ...  ...        ...        ...         ...   
Wolves   192   H     1     0  ...       4.25       5.75       13.25   
         256   H     1     0  ...       6.00       5.75       12.25   
         325   A     0     1  ...       7.00       3.25       14.00   
         331   A     0     1  ...       5.25       3.00       14.25   
         340   H     1     0  ...       6.00       6.50       10.75   

              H_R_AF_W_4  H_R_HY_W_4  H_R_AY_W_4  H_R_HR_W_4  H_R_AR_W_4  \
HomeTeam                                                                   
Arsenal  21        12.00        1.50        1.50        0.00        0.00   
         54        10.00        1.75        1.25        0.00        0.00   
         92        10.25        1.50        2.25        0.00        0.25   
         165        9.50        1.75        2.00        0.25        0.00   
         187       10.75        1.00        2.50        0.00        0.25   
...                  ...         ...         ...         ...         ...   
Wolves   192       11.25        2.75        2.50        0.00        0.00   
         256       11.50        3.25        3.50        0.25        0.25   
         325       10.50        2.75        3.50        0.25        0.25   
         331        9.75        2.75        3.50        0.25        0.00   
         340        9.75        3.00        2.25        0.00        0.25   

              H_R_FTR_code_W_4  H_R_HTR_code_W_4  
HomeTeam                                          
Arsenal  21               0.75              0.75  
         54               1.00              0.75  
         92               0.50              0.25  
         165              0.50              0.50  
         187              0.50              0.25  
...                        ...               ...  
Wolves   192              0.75              0.50  
         256             -0.50             -0.75  
         325             -0.50             -0.50  
         331             -0.50             -0.50  
         340              0.00             -0.25  

[200 rows x 164 columns]

In [14]:
home_with_rolling = grouped_home.reset_index(drop=True)
home_with_rolling


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,H_R_HC_W_4,H_R_AC_W_4,H_R_HF_W_4,H_R_AF_W_4,H_R_HY_W_4,H_R_AY_W_4,H_R_HR_W_4,H_R_AR_W_4,H_R_FTR_code_W_4,H_R_HTR_code_W_4
0,E0,26/08/2023,15:00,Arsenal,Fulham,2,2,D,0,1,...,7.50,3.25,11.75,12.00,1.50,1.50,0.00,0.00,0.75,0.75
1,E0,24/09/2023,14:00,Arsenal,Tottenham,2,2,D,1,1,...,6.75,2.50,10.75,10.00,1.75,1.25,0.00,0.00,1.00,0.75
2,E0,28/10/2023,15:00,Arsenal,Sheffield United,5,0,H,1,0,...,8.00,2.25,11.25,10.25,1.50,2.25,0.00,0.25,0.50,0.25
3,E0,17/12/2023,14:00,Arsenal,Brighton,2,0,H,0,0,...,6.25,4.00,10.25,9.50,1.75,2.00,0.25,0.00,0.50,0.50
4,E0,28/12/2023,20:15,Arsenal,West Ham,0,2,A,0,1,...,8.50,2.00,10.50,10.75,1.00,2.50,0.00,0.25,0.50,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,E0,30/12/2023,15:00,Wolves,Everton,3,0,H,1,0,...,4.25,5.75,13.25,11.25,2.75,2.50,0.00,0.00,0.75,0.50
196,E0,25/02/2024,13:30,Wolves,Sheffield United,1,0,H,1,0,...,6.00,5.75,12.25,11.50,3.25,3.50,0.25,0.25,-0.50,-0.75
197,E0,20/04/2024,19:30,Wolves,Arsenal,0,2,A,0,1,...,7.00,3.25,14.00,10.50,2.75,3.50,0.25,0.25,-0.50,-0.50
198,E0,24/04/2024,19:45,Wolves,Bournemouth,0,1,A,0,1,...,5.25,3.00,14.25,9.75,2.75,3.50,0.25,0.00,-0.50,-0.50


In [17]:
grouped_away = data.groupby("AwayTeam").apply(
    lambda x:
        rolling_averages_for_window_sizes(
            x,
            ROLLING_AVERAGES_COLUMNS,
            ROLLING_AVERAGES_WINDOW_SIZES,
            'A'
            )
        )
grouped_away

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_3727/1252556793.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_away = data.groupby("AwayTeam").apply(


Div        Date   Time        HomeTeam AwayTeam  FTHG  FTAG FTR  \
AwayTeam                                                                       
Arsenal  18   E0  21/08/2023  20:00  Crystal Palace  Arsenal     0     1   A   
         60   E0  30/09/2023  15:00     Bournemouth  Arsenal     0     4   A   
         86   E0  21/10/2023  17:30         Chelsea  Arsenal     2     2   D   
         126  E0  25/11/2023  17:30       Brentford  Arsenal     0     1   A   
         176  E0  23/12/2023  17:30       Liverpool  Arsenal     1     1   D   
...           ..         ...    ...             ...      ...   ...   ...  ..   
Wolves   183  E0  27/12/2023  19:30       Brentford   Wolves     1     4   A   
         207  E0  22/01/2024  19:45        Brighton   Wolves     0     0   D   
         243  E0  17/02/2024  15:00       Tottenham   Wolves     1     2   A   
         289  E0  30/03/2024  17:30     Aston Villa   Wolves     2     0   H   
         376  E0  19/05/2024  16:00       Liverpool   Wolves     2     0   H   

              HTHG  HTAG  ... A_R_HC_W_4 A_R_AC_W_4  A_R_HF_W_4  A_R_AF_W_4  \
AwayTeam                  ...                                                 
Arsenal  18      0     0  ...       2.50       6.50       10.50        8.25   
         60      0     2  ...       3.75       7.50       10.00       11.00   
         86      1     0  ...       1.75       6.75       12.75        9.25   
         126     0     0  ...       2.00       5.50       12.00       11.75   
         176     1     1  ...       1.25       7.00       11.75       10.75   
...            ...   ...  ...        ...        ...         ...         ...   
Wolves   183     1     3  ...       7.75       1.25       11.25       14.25   
         207     0     0  ...       7.25       6.00       12.25       12.25   
         243     0     1  ...       7.00       3.00       11.50       11.75   
         289     1     0  ...       7.75       1.25       10.00       14.50   
         376     2     0  ...       8.50       4.50       12.25       12.00   

              A_R_HY_W_4  A_R_AY_W_4  A_R_HR_W_4  A_R_AR_W_4  \
AwayTeam                                                       
Arsenal  18         1.50        1.50        0.00        0.00   
         60         1.25        2.00        0.00        0.00   
         86         2.00        1.25        0.00        0.25   
         126        2.50        2.75        0.00        0.25   
         176        2.25        1.75        0.00        0.25   
...                  ...         ...         ...         ...   
Wolves   183        1.50        3.00        0.00        0.25   
         207        1.50        2.25        0.25        0.25   
         243        1.50        3.25        0.00        0.00   
         289        1.50        2.25        0.00        0.25   
         376        1.75        2.00        0.00        0.00   

              A_R_FTR_code_W_4  A_R_HTR_code_W_4  
AwayTeam                                          
Arsenal  18              -1.00             -0.75  
         60              -0.75             -0.25  
         86              -1.00             -0.50  
         126             -0.50              0.00  
         176             -0.75              0.00  
...                        ...               ...  
Wolves   183              0.00              0.00  
         207              0.00              0.50  
         243              0.75              0.25  
         289             -0.25             -0.25  
         376              0.25              0.00  

[200 rows x 164 columns]

In [18]:
away_with_rolling = grouped_away.reset_index(drop=True)
away_with_rolling

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,A_R_HC_W_4,A_R_AC_W_4,A_R_HF_W_4,A_R_AF_W_4,A_R_HY_W_4,A_R_AY_W_4,A_R_HR_W_4,A_R_AR_W_4,A_R_FTR_code_W_4,A_R_HTR_code_W_4
0,E0,21/08/2023,20:00,Crystal Palace,Arsenal,0,1,A,0,0,...,2.50,6.50,10.50,8.25,1.50,1.50,0.00,0.00,-1.00,-0.75
1,E0,30/09/2023,15:00,Bournemouth,Arsenal,0,4,A,0,2,...,3.75,7.50,10.00,11.00,1.25,2.00,0.00,0.00,-0.75,-0.25
2,E0,21/10/2023,17:30,Chelsea,Arsenal,2,2,D,1,0,...,1.75,6.75,12.75,9.25,2.00,1.25,0.00,0.25,-1.00,-0.50
3,E0,25/11/2023,17:30,Brentford,Arsenal,0,1,A,0,0,...,2.00,5.50,12.00,11.75,2.50,2.75,0.00,0.25,-0.50,0.00
4,E0,23/12/2023,17:30,Liverpool,Arsenal,1,1,D,1,1,...,1.25,7.00,11.75,10.75,2.25,1.75,0.00,0.25,-0.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,E0,27/12/2023,19:30,Brentford,Wolves,1,4,A,1,3,...,7.75,1.25,11.25,14.25,1.50,3.00,0.00,0.25,0.00,0.00
196,E0,22/01/2024,19:45,Brighton,Wolves,0,0,D,0,0,...,7.25,6.00,12.25,12.25,1.50,2.25,0.25,0.25,0.00,0.50
197,E0,17/02/2024,15:00,Tottenham,Wolves,1,2,A,0,1,...,7.00,3.00,11.50,11.75,1.50,3.25,0.00,0.00,0.75,0.25
198,E0,30/03/2024,17:30,Aston Villa,Wolves,2,0,H,1,0,...,7.75,1.25,10.00,14.50,1.50,2.25,0.00,0.25,-0.25,-0.25


In [19]:
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HomeTeamCode,AwayTeamCode,FTR_code,HTR_code
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,1.95,1.97,NaN,NaN,1.92,1.95,5,12,-1,-1
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,1.93,1.97,2.01,2.09,1.95,1.92,0,15,1,1
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.01,1.92,2.06,1.96,1.96,1.91,2,18,0,0
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.00,1.91,2.14,1.93,2.00,1.86,4,11,1,1
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,2.04,1.88,2.08,1.99,1.98,1.88,8,9,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,19/05/2024,16:00,Crystal Palace,Aston Villa,5,0,H,2,0,...,1.78,2.16,1.93,2.21,1.81,2.05,7,1,1,1
376,E0,19/05/2024,16:00,Liverpool,Wolves,2,0,H,2,0,...,2.04,1.85,2.10,1.89,2.04,1.82,10,19,1,1
377,E0,19/05/2024,16:00,Luton,Fulham,2,4,A,1,2,...,1.99,1.93,2.02,1.94,1.96,1.91,11,9,-1,-1
378,E0,19/05/2024,16:00,Man City,West Ham,3,1,H,2,1,...,1.99,1.90,2.05,1.99,1.96,1.91,12,18,1,1


In [26]:
data_with_home_rolling = calculate_rolling_averages_home_team(data)
data_with_home_rolling

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,H_R_HC_W_4,H_R_AC_W_4,H_R_HF_W_4,H_R_AF_W_4,H_R_HY_W_4,H_R_AY_W_4,H_R_HR_W_4,H_R_AR_W_4,H_R_FTR_code_W_4,H_R_HTR_code_W_4
112,E0,10/12/2023,14:00,Luton,Man City,1,2,A,1,0,...,7.00,4.25,11.00,10.00,1.25,1.00,0.00,0.25,-0.50,-0.50
55,E0,12/01/2024,19:45,Burnley,Luton,1,1,D,1,0,...,7.25,5.00,9.75,9.00,1.00,2.75,0.25,0.00,-1.00,-0.75
43,E0,12/11/2023,14:00,Brighton,Sheffield United,1,1,D,1,0,...,7.25,3.00,10.50,13.50,1.75,3.00,0.00,0.00,0.50,0.00
58,E0,13/04/2024,15:00,Burnley,Brighton,1,1,D,0,0,...,6.75,4.25,9.50,10.00,0.75,2.25,0.25,0.00,-0.75,-0.25
60,E0,13/08/2023,16:30,Chelsea,Liverpool,1,1,D,1,1,...,4.25,4.25,12.25,10.25,3.50,2.00,0.00,0.00,0.75,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,E0,31/01/2024,19:30,Tottenham,Brentford,3,2,H,0,1,...,8.00,5.00,12.50,15.00,2.25,3.25,0.00,0.50,0.00,-0.50
108,E0,31/03/2024,14:00,Liverpool,Brighton,2,1,H,1,1,...,6.75,3.25,12.25,14.00,1.50,3.50,0.00,0.00,0.75,0.50
128,E0,31/03/2024,16:30,Man City,Arsenal,0,0,D,0,0,...,8.50,4.25,5.50,11.75,1.75,3.25,0.25,0.00,0.75,1.00
94,E0,31/12/2023,14:00,Fulham,Arsenal,2,1,H,1,1,...,8.50,5.75,8.75,14.50,1.25,1.75,0.00,0.00,0.00,0.00


In [28]:
data_with_home_rolling_after_new_rolling = data.groupby("AwayTeam").apply(
    lambda x:
        rolling_averages_for_window_sizes(
            x,
            ROLLING_AVERAGES_COLUMNS,
            ROLLING_AVERAGES_WINDOW_SIZES,
            'A'
            )
        )
data_with_home_rolling_after_new_rolling

/var/folders/l3/bpd7p2510r9bclrfy43fqh0w0000gn/T/ipykernel_3727/3060801427.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_with_home_rolling_after_new_rolling = data.groupby("AwayTeam").apply(


Div        Date   Time        HomeTeam AwayTeam  FTHG  FTAG FTR  \
AwayTeam                                                                       
Arsenal  18   E0  21/08/2023  20:00  Crystal Palace  Arsenal     0     1   A   
         60   E0  30/09/2023  15:00     Bournemouth  Arsenal     0     4   A   
         86   E0  21/10/2023  17:30         Chelsea  Arsenal     2     2   D   
         126  E0  25/11/2023  17:30       Brentford  Arsenal     0     1   A   
         176  E0  23/12/2023  17:30       Liverpool  Arsenal     1     1   D   
...           ..         ...    ...             ...      ...   ...   ...  ..   
Wolves   183  E0  27/12/2023  19:30       Brentford   Wolves     1     4   A   
         207  E0  22/01/2024  19:45        Brighton   Wolves     0     0   D   
         243  E0  17/02/2024  15:00       Tottenham   Wolves     1     2   A   
         289  E0  30/03/2024  17:30     Aston Villa   Wolves     2     0   H   
         376  E0  19/05/2024  16:00       Liverpool   Wolves     2     0   H   

              HTHG  HTAG  ... A_R_HC_W_4 A_R_AC_W_4  A_R_HF_W_4  A_R_AF_W_4  \
AwayTeam                  ...                                                 
Arsenal  18      0     0  ...       2.50       6.50       10.50        8.25   
         60      0     2  ...       3.75       7.50       10.00       11.00   
         86      1     0  ...       1.75       6.75       12.75        9.25   
         126     0     0  ...       2.00       5.50       12.00       11.75   
         176     1     1  ...       1.25       7.00       11.75       10.75   
...            ...   ...  ...        ...        ...         ...         ...   
Wolves   183     1     3  ...       7.75       1.25       11.25       14.25   
         207     0     0  ...       7.25       6.00       12.25       12.25   
         243     0     1  ...       7.00       3.00       11.50       11.75   
         289     1     0  ...       7.75       1.25       10.00       14.50   
         376     2     0  ...       8.50       4.50       12.25       12.00   

              A_R_HY_W_4  A_R_AY_W_4  A_R_HR_W_4  A_R_AR_W_4  \
AwayTeam                                                       
Arsenal  18         1.50        1.50        0.00        0.00   
         60         1.25        2.00        0.00        0.00   
         86         2.00        1.25        0.00        0.25   
         126        2.50        2.75        0.00        0.25   
         176        2.25        1.75        0.00        0.25   
...                  ...         ...         ...         ...   
Wolves   183        1.50        3.00        0.00        0.25   
         207        1.50        2.25        0.25        0.25   
         243        1.50        3.25        0.00        0.00   
         289        1.50        2.25        0.00        0.25   
         376        1.75        2.00        0.00        0.00   

              A_R_FTR_code_W_4  A_R_HTR_code_W_4  
AwayTeam                                          
Arsenal  18              -1.00             -0.75  
         60              -0.75             -0.25  
         86              -1.00             -0.50  
         126             -0.50              0.00  
         176             -0.75              0.00  
...                        ...               ...  
Wolves   183              0.00              0.00  
         207              0.00              0.50  
         243              0.75              0.25  
         289             -0.25             -0.25  
         376              0.25              0.00  

[200 rows x 164 columns]

In [29]:
data_with_home_away_rolling = calculate_rolling_averages_away_team(data_with_home_rolling)
data_with_home_away_rolling

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,A_R_HC_W_4,A_R_AC_W_4,A_R_HF_W_4,A_R_AF_W_4,A_R_HY_W_4,A_R_AY_W_4,A_R_HR_W_4,A_R_AR_W_4,A_R_FTR_code_W_4,A_R_HTR_code_W_4
0,E0,30/09/2023,15:00,Bournemouth,Arsenal,0,4,A,0,2,...,3.75,7.50,10.00,11.00,1.25,2.00,0.00,0.00,-0.75,-0.25
1,E0,31/03/2024,16:30,Man City,Arsenal,0,0,D,0,0,...,4.25,7.75,9.50,10.00,0.75,1.00,0.00,0.00,-1.00,-0.50
2,E0,31/12/2023,14:00,Fulham,Arsenal,2,1,H,1,1,...,5.75,6.75,9.50,13.25,0.50,1.25,0.00,0.00,-0.75,-0.50
3,E0,27/08/2023,14:00,Burnley,Aston Villa,1,3,A,0,2,...,4.75,6.00,10.00,10.50,1.00,3.50,0.00,0.00,0.00,0.00
4,E0,31/01/2024,19:30,Tottenham,Brentford,3,2,H,0,1,...,6.00,4.50,10.25,9.50,0.75,2.25,0.00,0.00,0.50,0.50
5,E0,30/01/2024,19:45,Luton,Brighton,4,0,H,3,0,...,4.75,2.25,11.75,9.75,2.75,2.50,0.25,0.25,0.25,0.50
6,E0,30/09/2023,12:30,Aston Villa,Brighton,6,1,H,3,0,...,6.00,2.75,12.75,10.50,2.75,2.50,0.00,0.25,0.25,0.50
7,E0,31/03/2024,14:00,Liverpool,Brighton,2,1,H,1,1,...,5.50,3.25,14.50,13.00,2.50,3.00,0.00,0.25,0.50,0.50
8,E0,30/12/2023,15:00,Aston Villa,Burnley,3,2,H,2,1,...,8.25,4.25,8.75,11.25,1.50,2.75,0.00,0.25,0.50,0.50
9,E0,31/01/2024,19:30,Man City,Burnley,3,1,H,2,0,...,8.25,3.50,9.25,12.00,2.25,2.25,0.00,0.50,0.75,0.75
